**Unique Index Calculation for 2D Grid**

In [38]:
%%writefile one.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>

__global__ void unique_gid_calc_2d(int *data){
  //int tid = threadIdx.x;
  //int offset = blockIdx.x * blockDim.x; // this one is for one dimensional

  int row_offset = blockDim.x * gridDim.x + blockIdx.x;
  int block_offset = blockDim.x * blockIdx.x;
  int tid = threadIdx.x;

  int gid =  row_offset + block_offset + tid;
  printf("blockIdx.x : %d, blockIdx.y : %d, threadIdx.x : %d, gid: %d - data : %d \n", blockIdx.x, blockIdx.y, threadIdx.x, gid, data[gid]);
}

int main(){
  int array_size = 16;
  int array_byt_size = sizeof(int) * array_size;
  int h_data[] = {13,21,34,51,23,31,19,12,32,91,43,44,54,45,15,29};

  int * d_data;
  cudaMalloc(&d_data, array_byt_size);
  cudaMemcpy(d_data, h_data, array_byt_size, cudaMemcpyHostToDevice);

  dim3 block(4);
  dim3 grid(2,2);

  unique_gid_calc_2d<<<grid, block>>>(d_data);
  cudaDeviceSynchronize();

  cudaDeviceReset();
  return 0;
}

Overwriting one.cu


In [39]:
!nvcc -arch=sm_75 one.cu -o one

In [40]:
!./one

blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 0, gid: 13 - data : 45 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 1, gid: 14 - data : 15 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 2, gid: 15 - data : 29 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 3, gid: 16 - data : 0 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 0, gid: 8 - data : 32 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 1, gid: 9 - data : 91 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 2, gid: 10 - data : 43 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 3, gid: 11 - data : 44 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 0, gid: 13 - data : 45 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 1, gid: 14 - data : 15 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 2, gid: 15 - data : 29 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 3, gid: 16 - data : 0 
blockIdx.x : 0, blockIdx.y : 0, threadIdx.x : 0, gid: 8 - data : 32 
blockIdx.x : 0, blockIdx.y : 0, threadIdx.x : 1, gid: 9 - data : 91 
blockIdx.x : 0, blockIdx.y

In [41]:
%%writefile two.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>

__global__ void unique_gid_calc_2d(int *data){
  int row_offset = (blockDim.x * blockDim.y * gridDim.x) * blockIdx.y;
  int block_offset = (blockDim.x * blockDim.y) * blockIdx.x;
  int tid = threadIdx.y * blockDim.x + threadIdx.x;

  int gid =  row_offset + block_offset + tid;
  printf("blockIdx.x : %d, blockIdx.y : %d, threadIdx.x : %d, gid: %d - data : %d \n", blockIdx.x, blockIdx.y, threadIdx.x, gid, data[gid]);
}

int main(){
  int array_size = 16;
  int array_byt_size = sizeof(int) * array_size;
  int h_data[] = {13,21,34,51,23,31,19,12,32,91,43,44,54,45,15,29};

  int * d_data;
  cudaMalloc(&d_data, array_byt_size);
  cudaMemcpy(d_data, h_data, array_byt_size, cudaMemcpyHostToDevice);

  dim3 block(2,2);
  dim3 grid(2,2);

  unique_gid_calc_2d<<<grid, block>>>(d_data);
  cudaDeviceSynchronize();

  cudaDeviceReset();
  return 0;
}

Overwriting two.cu


In [42]:
!nvcc -arch=sm_75 two.cu -o two

In [43]:
!./two

blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 0, gid: 4 - data : 23 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 1, gid: 5 - data : 31 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 0, gid: 6 - data : 19 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 1, gid: 7 - data : 12 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 0, gid: 8 - data : 32 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 1, gid: 9 - data : 91 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 0, gid: 10 - data : 43 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 1, gid: 11 - data : 44 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 0, gid: 12 - data : 54 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 1, gid: 13 - data : 45 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 0, gid: 14 - data : 15 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 1, gid: 15 - data : 29 
blockIdx.x : 0, blockIdx.y : 0, threadIdx.x : 0, gid: 0 - data : 13 
blockIdx.x : 0, blockIdx.y : 0, threadIdx.x : 1, gid: 1 - data : 21 
blockIdx.x : 0, blockIdx.y :

In [44]:
%%writefile three.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>

__global__ void unique_gid_calc_2d(int *data){
  // Correct 2D global ID calculation
  int gid = blockIdx.y * gridDim.x * blockDim.x + blockIdx.x * blockDim.x + threadIdx.x;

  printf("blockIdx.x : %d, blockIdx.y : %d, threadIdx.x : %d, gid: %d - data : %d \n",
         blockIdx.x, blockIdx.y, threadIdx.x, gid, data[gid]);
}

int main(){
  int array_size = 16;
  int array_byt_size = sizeof(int) * array_size;
  int h_data[] = {13,21,34,51,23,31,19,12,32,91,43,44,54,45,15,29};

  int * d_data;
  cudaMalloc(&d_data, array_byt_size);
  cudaMemcpy(d_data, h_data, array_byt_size, cudaMemcpyHostToDevice);

  dim3 block(4);  // 4 threads per block
  dim3 grid(2,2); // 2x2 grid of blocks = 4 blocks total

  unique_gid_calc_2d<<<grid, block>>>(d_data);
  cudaDeviceSynchronize();

  cudaFree(d_data);
  cudaDeviceReset();
  return 0;
}

Writing three.cu


In [45]:
!nvcc -arch=sm_75 three.cu -o three

In [46]:
!./three

blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 0, gid: 4 - data : 23 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 1, gid: 5 - data : 31 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 2, gid: 6 - data : 19 
blockIdx.x : 1, blockIdx.y : 0, threadIdx.x : 3, gid: 7 - data : 12 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 0, gid: 8 - data : 32 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 1, gid: 9 - data : 91 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 2, gid: 10 - data : 43 
blockIdx.x : 0, blockIdx.y : 1, threadIdx.x : 3, gid: 11 - data : 44 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 0, gid: 12 - data : 54 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 1, gid: 13 - data : 45 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 2, gid: 14 - data : 15 
blockIdx.x : 1, blockIdx.y : 1, threadIdx.x : 3, gid: 15 - data : 29 
blockIdx.x : 0, blockIdx.y : 0, threadIdx.x : 0, gid: 0 - data : 13 
blockIdx.x : 0, blockIdx.y : 0, threadIdx.x : 1, gid: 1 - data : 21 
blockIdx.x : 0, blockIdx.y :